# SailGP Data Analyst Challenge

The aim is to test you python abilities. The challenge is to analyze the data provided and answer the questions below. You can use any library you want to help you with the analysis. The data is from the SailGP event in Auckland 2025. The data is in the 'DATA' folder.

There are various sources available.

The Boat Logs are in the 'Boat_Logs' folder. The data is in csv format and the columns are described in the 'Boat_Logs/Boat_Logs_Columns.csv' file.
The 'Course_Marks_2025-01-19.csv' file contains the mark positions and wind reading on the course for the whole day.

The Race_XML folder contains the xml files for each race that contains information on where the boundaries of the course are, the theoretical position of the marks and the target racecourse axis.

The 2025-01-19_man_summary.csv file contains the metrics from the manoeuvre summary for the day.
The 2025-01-19_straight_lines.csv file contains the metrics from the straight line summary for the day.

Both are derived from the boat logs.

The 2502 m8_APW_HSB2_HSRW.kph.csv file contains the polar data for the boats in that config.

## Requierements
- Chose at least 3 questions from the list below to answer.
- Python 3.8 or higher
- Notebook should be able to run without any errors from start to finish.
- Specify the libraries (imports) used in the notebook.
- Any comments to make the notebook self-explanatory and easy to follow would be appreciated.
- If you can't get to the end of a question, we would appreciate the code you have written so far and explain what you were trying to do.

## Further information:
- We usually use bokeh for visualizations. So any showcase of bokeh would be appreciated.

## Submitting the results.
It would be great if you could provide a jupyter notebook with the code and the results of the analysis. You can submit the results by sharing a link to a git repository.


### Imports and re-used functions
Free section to initialize the notebook with the necessary imports and functions that will be used in the notebook.



In [141]:
# Data manipulation
import pandas as pd
import numpy as np

# Visualization
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool, ColumnDataSource, Span, Label
from bokeh.layouts import row, column, gridplot
from bokeh.palettes import Category10
import matplotlib.pyplot as plt

# Machine Learning (for Question 7)
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

# XML parsing
from bs4 import BeautifulSoup
import xml.etree.ElementTree as ET

# Scientific computing
from scipy import stats
from scipy.spatial.distance import cdist

# Utilities
import os
from pathlib import Path
from datetime import datetime, timedelta
import math

# Configure Bokeh for notebook output
output_notebook()

# Circular Mean Function (for compass directions)
def circular_mean(angles, weights=None):
    # Convert to numpy array and remove NaN values
    angles_array = np.asarray(angles)
    valid_mask = ~np.isnan(angles_array)
    
    # If no valid angles, return NaN
    if not np.any(valid_mask):
        return np.nan
    
    # Filter to only valid angles
    valid_angles = angles_array[valid_mask]
    
    # Double-check we have at least one valid angle
    if len(valid_angles) == 0:
        return np.nan
    
    # Convert to radians for trigonometric functions
    angles_rad = np.deg2rad(valid_angles)
    
    # Calculate mean of unit vectors (sin and cos components)
    if weights is not None:
        # Filter weights to match valid angles
        weights_array = np.asarray(weights)
        if len(weights_array) != len(angles_array):
            # Weights don't match angles array length
            return np.nan
        valid_weights = weights_array[valid_mask]
        # Normalize weights to sum to 1
        weight_sum = np.sum(valid_weights)
        if weight_sum <= 0 or np.isnan(weight_sum):
            return np.nan
        valid_weights = valid_weights / weight_sum
        sin_mean = np.average(np.sin(angles_rad), weights=valid_weights)
        cos_mean = np.average(np.cos(angles_rad), weights=valid_weights)
    else:
        sin_mean = np.mean(np.sin(angles_rad))
        cos_mean = np.mean(np.cos(angles_rad))
    
    # Check if both sin_mean and cos_mean are valid (not NaN)
    if np.isnan(sin_mean) or np.isnan(cos_mean):
        return np.nan
    
    # Convert mean vector back to angle using arctan2
    # arctan2 handles the case where both are zero (returns 0)
    mean_rad = np.arctan2(sin_mean, cos_mean)
    mean_deg = np.rad2deg(mean_rad)
    
    # Check if mean_deg is NaN before doing modulo operation
    # This prevents RuntimeWarning when mean_deg is NaN
    if np.isnan(mean_deg):
        return np.nan
    
    # Normalize to 0-360 range (arctan2 returns -180 to 180)
    # Modulo operation ensures result is always positive
    return mean_deg % 360


def downsample_compass_direction(df, direction_col, time_col, window_seconds):
    df = df.copy()
    df[time_col] = pd.to_datetime(df[time_col])
    df = df.set_index(time_col)
    
    # Resample using circular mean
    # The lambda function handles NaN values - circular_mean will return NaN
    # if all values in a group are NaN
    resampled = df.groupby(pd.Grouper(freq=f'{window_seconds}S')).apply(
        lambda x: circular_mean(x[direction_col].values) if len(x) > 0 else np.nan
    )
    
    result = pd.DataFrame({
        'time': resampled.index,
        f'{direction_col}_mean': resampled.values
    })
    
    # Remove rows where the result is NaN (optional - comment out if you want to keep them)
    # result = result.dropna()
    
    return result


# XML Parser Function (for course XML files)
def parse_course_xml(xml_path):
    tree = ET.parse(xml_path)
    root = tree.getroot()
    
    result = {
        'marks': [],
        'boundaries': {},
        'mark_sequence': [],
        'race_info': {}
    }
    
    # Extract race info
    race_id = root.find('RaceID')
    race_start = root.find('RaceStartTime')
    if race_id is not None:
        result['race_info']['race_id'] = race_id.text
    if race_start is not None:
        result['race_info']['start_time'] = race_start.get('Start')
    
    # Extract marks
    course = root.find('Course')
    if course is not None:
        for compound_mark in course.findall('CompoundMark'):
            compound_id = compound_mark.get('CompoundMarkID')
            compound_name = compound_mark.get('Name')
            
            for mark in compound_mark.findall('Mark'):
                result['marks'].append({
                    'compound_id': compound_id,
                    'compound_name': compound_name,
                    'seq_id': mark.get('SeqID'),
                    'name': mark.get('Name'),
                    'lat': float(mark.get('TargetLat')),
                    'lon': float(mark.get('TargetLng')),
                    'source_id': mark.get('SourceID')
                })
    
    # Extract mark sequence
    sequence = root.find('CompoundMarkSequence')
    if sequence is not None:
        for corner in sequence.findall('Corner'):
            result['mark_sequence'].append({
                'seq_id': corner.get('SeqID'),
                'compound_mark_id': corner.get('CompoundMarkID'),
                'rounding': corner.get('Rounding'),
                'zone_size': corner.get('ZoneSize')
            })
    
    # Extract boundaries
    for course_limit in root.findall('CourseLimit'):
        limit_name = course_limit.get('name')
        limit_points = []
        
        for limit in course_limit.findall('Limit'):
            limit_points.append({
                'lat': float(limit.get('Lat')),
                'lon': float(limit.get('Lon'))
            })
        
        if limit_points:
            result['boundaries'][limit_name] = limit_points
    
    return result


# Data Loading Helper
def load_boat_logs(data_dir='Data'):
    boat_logs_dir = Path(data_dir) / 'Boat_logs'
    boat_logs = {}
    
    for csv_file in boat_logs_dir.glob('data_*.csv'):
        country_code = csv_file.stem.split('_')[1]
        boat_logs[country_code] = pd.read_csv(csv_file)
    
    return boat_logs


def load_course_marks(data_dir='Data', filename='Course_Marks_2025-01-19.csv'):
    filepath = Path(data_dir) / filename
    return pd.read_csv(filepath)


def load_summary_data(data_dir='Data', summary_type='man'):
    if summary_type == 'man':
        filename = '2025-01-19_man_summary.csv'
    elif summary_type == 'straight':
        filename = '2025-01-19_straight_lines.csv'
    else:
        raise ValueError("summary_type must be 'man' or 'straight'")
    
    filepath = Path(data_dir) / filename
    return pd.read_csv(filepath)


def load_polar_data(data_dir='Data', filename='2502 m8_APW_HSB2_HSRW.kph.csv'):
    filepath = Path(data_dir) / filename
    return pd.read_csv(filepath)


# Distance Calculation (Haversine formula)
def haversine_distance(lat1, lon1, lat2, lon2):
    # Earth radius in meters
    R = 6371000
    
    # Convert to radians
    phi1 = np.deg2rad(lat1)
    phi2 = np.deg2rad(lat2)
    delta_phi = np.deg2rad(lat2 - lat1)
    delta_lambda = np.deg2rad(lon2 - lon1)
    
    # Haversine formula
    a = np.sin(delta_phi/2)**2 + np.cos(phi1) * np.cos(phi2) * np.sin(delta_lambda/2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    
    distance = R * c
    
    return distance


def bearing(lat1, lon1, lat2, lon2):
    lat1_rad = np.deg2rad(lat1)
    lat2_rad = np.deg2rad(lat2)
    delta_lon = np.deg2rad(lon2 - lon1)
    
    y = np.sin(delta_lon) * np.cos(lat2_rad)
    x = np.cos(lat1_rad) * np.sin(lat2_rad) - np.sin(lat1_rad) * np.cos(lat2_rad) * np.cos(delta_lon)
    
    bearing_rad = np.arctan2(y, x)
    bearing_deg = np.rad2deg(bearing_rad)
    
    return (bearing_deg + 360) % 360

# Coordinate Utilities
def calculate_vmc(lat, lon, target_lat, target_lon, boat_speed, heading):
    # Calculate bearing to target
    target_bearing = np.array([bearing(lat[i], lon[i], target_lat, target_lon) 
                               for i in range(len(lat))])
    
    # Calculate angle between heading and target bearing
    angle_diff = target_bearing - heading
    # Normalize to -180 to 180
    angle_diff = ((angle_diff + 180) % 360) - 180
    
    # VMC = speed * cos(angle difference)
    vmc = boat_speed * np.cos(np.deg2rad(angle_diff))
    
    return vmc


def calculate_distance_to_leader(boat_positions, leader_positions):
    distances = {}
    
    for boat_id, boat_df in boat_positions.items():
        # Align timestamps if needed (assuming same time index)
        if len(boat_df) == len(leader_positions):
            dist = haversine_distance(
                boat_df['lat'].values,
                boat_df['lon'].values,
                leader_positions['lat'].values,
                leader_positions['lon'].values
            )
            distances[boat_id] = dist
    
    return distances

# Visualization Helpers
def setup_bokeh_plot(title, x_axis_label, y_axis_label, width=800, height=400):
    p = figure(
        title=title,
        x_axis_label=x_axis_label,
        y_axis_label=y_axis_label,
        width=width,
        height=height,
        tools='pan,box_zoom,wheel_zoom,reset,save'
    )
    
    return p

def plot_trajectory_bokeh(lat, lon, title='Boat Trajectory', color='blue', size=2):
    p = setup_bokeh_plot(title, 'Longitude', 'Latitude')
    
    source = ColumnDataSource(data={
        'lat': lat,
        'lon': lon
    })
    
    p.line('lon', 'lat', source=source, line_width=2, color=color, legend_label='Trajectory')
    p.circle('lon', 'lat', source=source, size=size, color=color, alpha=0.6)
    
    p.add_tools(HoverTool(tooltips=[('Lat', '@lat'), ('Lon', '@lon')]))
    
    return p

# Print confirmation
print(" Imports loaded successfully!")

Loading BokehJS ...

 Imports loaded successfully!


## Question 1: Write a Python function that can take a compass direction (ie. TWD or Heading) and calculate an accurate mean value across a downsampled frequency. Eg. If TWD is at 1Hz, give me a 10s average.

Compass directions are circular (0-360°), so we need to use circular statistics to calculate accurate means. A simple arithmetic mean would fail for directions near 0°/360°.

In [142]:
def downsample_compass_mean(directions, time_index, window_seconds):
    # Convert to DataFrame for easier manipulation
    df = pd.DataFrame({
        'direction': directions,
        'time': pd.to_datetime(time_index)
    })
    df = df.set_index('time')
    
    # Circular mean: convert to unit vectors, average, convert back to angle
    def circular_mean_group(group):
        angles = group['direction'].dropna()
        if len(angles) == 0:
            return np.nan
        
        # Convert to radians and calculate mean of unit vectors
        angles_rad = np.deg2rad(angles)
        sin_mean = np.sin(angles_rad).mean()
        cos_mean = np.cos(angles_rad).mean()
        
        # Convert back to degrees and normalize to 0-360
        mean_rad = np.arctan2(sin_mean, cos_mean)
        mean_deg = np.rad2deg(mean_rad)
        return mean_deg % 360
    
    # Resample using the circular mean function
    resampled = df.resample(f'{window_seconds}S').apply(circular_mean_group)
    
    # Return as DataFrame
    result = pd.DataFrame({
        'time': resampled.index,
        'mean_direction': resampled.values
    })
    
    return result


# Example usage
boat_logs = load_boat_logs(data_dir='Data')
boat_data = boat_logs['BRA'].copy()

# Downsample TWD from 1Hz to 10-second averages
twd_downsampled = downsample_compass_mean(
    boat_data['TWD_SGP_deg'], 
    boat_data['DATETIME'], 
    10
)

# Downsample Heading from 1Hz to 10-second averages
heading_downsampled = downsample_compass_mean(
    boat_data['HEADING_deg'], 
    boat_data['DATETIME'], 
    10
)

print(f"Original data: {len(boat_data)} points at 1Hz")
print(f"Downsampled TWD: {len(twd_downsampled)} points (10s averages)")
print(f"Downsampled Heading: {len(heading_downsampled)} points (10s averages)")
print(f"\nExample TWD values:")
print(twd_downsampled.head(10))

# Visualization: Compare original vs downsampled data using Bokeh
# Prepare time data for plotting
boat_data['DATETIME'] = pd.to_datetime(boat_data['DATETIME'])
boat_data_sorted = boat_data.sort_values('DATETIME').reset_index(drop=True)
boat_data_sorted['time_seconds'] = (boat_data_sorted['DATETIME'] - boat_data_sorted['DATETIME'].iloc[0]).dt.total_seconds()

# Merge downsampled data with original for comparison
twd_downsampled['time'] = pd.to_datetime(twd_downsampled['time'])
twd_downsampled['time_seconds'] = (twd_downsampled['time'] - boat_data_sorted['DATETIME'].iloc[0]).dt.total_seconds()

# Create Bokeh visualization
p1 = figure(
    title='True Wind Direction (TWD) - Original vs 10s Downsampled',
    x_axis_label='Time (seconds)',
    y_axis_label='TWD (degrees)',
    width=1000,
    height=400,
    tools='pan,box_zoom,wheel_zoom,reset,save,hover'
)

# Plot original data (sample every 10th point for performance)
sample_indices = np.arange(0, len(boat_data_sorted), 10)
source_original = ColumnDataSource({
    'time': boat_data_sorted.loc[sample_indices, 'time_seconds'],
    'twd': boat_data_sorted.loc[sample_indices, 'TWD_SGP_deg']
})
p1.circle('time', 'twd', source=source_original, size=2, color='lightblue', 
          alpha=0.3, legend_label='Original (1Hz, sampled)')

# Plot downsampled data
source_downsampled = ColumnDataSource(twd_downsampled)
p1.line('time_seconds', 'mean_direction', source=source_downsampled, 
        line_width=3, color='blue', legend_label='10s Average (Circular Mean)')
p1.circle('time_seconds', 'mean_direction', source=source_downsampled, 
          size=6, color='blue', alpha=0.7)

p1.legend.location = 'top_left'
p1.grid.grid_line_alpha = 0.3
p1.add_tools(HoverTool(tooltips=[
    ('Time', '@time_seconds{0.0}s'),
    ('TWD', '@mean_direction{0.1f}°')
], mode='vline'))
show(p1)

# Visualization for Heading
heading_downsampled['time'] = pd.to_datetime(heading_downsampled['time'])
heading_downsampled['time_seconds'] = (heading_downsampled['time'] - boat_data_sorted['DATETIME'].iloc[0]).dt.total_seconds()

p2 = figure(
    title='Boat Heading - Original vs 10s Downsampled',
    x_axis_label='Time (seconds)',
    y_axis_label='Heading (degrees)',
    width=1000,
    height=400,
    tools='pan,box_zoom,wheel_zoom,reset,save,hover'
)

# Plot original data (sample every 10th point for performance)
source_original_hdg = ColumnDataSource({
    'time': boat_data_sorted.loc[sample_indices, 'time_seconds'],
    'heading': boat_data_sorted.loc[sample_indices, 'HEADING_deg']
})
p2.circle('time', 'heading', source=source_original_hdg, size=2, color='lightcoral', 
          alpha=0.3, legend_label='Original (1Hz, sampled)')

# Plot downsampled data
source_downsampled_hdg = ColumnDataSource(heading_downsampled)
p2.line('time_seconds', 'mean_direction', source=source_downsampled_hdg, 
        line_width=3, color='red', legend_label='10s Average (Circular Mean)')
p2.circle('time_seconds', 'mean_direction', source=source_downsampled_hdg, 
          size=6, color='red', alpha=0.7)

p2.legend.location = 'top_left'
p2.grid.grid_line_alpha = 0.3
p2.add_tools(HoverTool(tooltips=[
    ('Time', '@time_seconds{0.0}s'),
    ('Heading', '@mean_direction{0.1f}°')
], mode='vline'))
show(p2)


Original data: 2132 points at 1Hz
Downsampled TWD: 325 points (10s averages)
Downsampled Heading: 325 points (10s averages)

Example TWD values:
                 time  mean_direction
0 2025-01-19 03:06:00       57.586000
1 2025-01-19 03:06:10       59.377023
2 2025-01-19 03:06:20       60.783002
3 2025-01-19 03:06:30       58.128148
4 2025-01-19 03:06:40       56.917022
5 2025-01-19 03:06:50       61.110989
6 2025-01-19 03:07:00       60.940998
7 2025-01-19 03:07:10       60.783974
8 2025-01-19 03:07:20       63.389001
9 2025-01-19 03:07:30       62.097999


## Question 2: Given a course XML and a timeseries of boat Lat/Lon values, calculate a VMC column for the same timeseries.


In [143]:
# Calculate VMC for one boat in one race
# Select one race XML file (using the first one)
race_xml_dir = Path('Data/Race_XMLs')
race_xml_files = sorted(race_xml_dir.glob('*.xml'))
xml_path = race_xml_files[0]  # Use first race
race_number = int(xml_path.stem.split('_')[0])
print(f"Processing Race {race_number}")

# Load course XML and extract waypoints
course_data = parse_course_xml(xml_path)
mark_sequence = course_data['mark_sequence']
marks_by_compound = {m['compound_id']: m for m in course_data['marks']}
waypoints = []
for corner in mark_sequence:
    compound_id = corner['compound_mark_id']
    mark = marks_by_compound.get(compound_id)
    if mark:
        waypoints.append({
            'name': mark['name'],
            'lat': mark['lat'],
            'lon': mark['lon']
        })

print(f"Course has {len(waypoints)} waypoints: {[w['name'] for w in waypoints]}")

# Select one boat (e.g., BRA)
boat_code = 'BRA'
boat_logs = load_boat_logs(data_dir='Data')
boat_data = boat_logs[boat_code].copy()
boat_data['DATETIME'] = pd.to_datetime(boat_data['DATETIME'])

# Filter to this race only
boat_data = boat_data[boat_data['TRK_RACE_NUM_unk'] == race_number].copy()
print(f"\nBoat {boat_code}: {len(boat_data)} data points for race {race_number}")

# Calculate VMC for each data point
boat_data['VMC_km_h'] = np.nan
boat_data['target_waypoint'] = ''
current_waypoint_idx = 0
prev_dist = None

for idx, row in boat_data.iterrows():
    lat = row['LATITUDE_GPS_unk']
    lon = row['LONGITUDE_GPS_unk']
    speed = row['BOAT_SPEED_km_h_1']
    heading = row['HEADING_deg']
    
    # Skip if speed or heading missing
    if pd.isna(speed) or pd.isna(heading):
        continue
    
    if current_waypoint_idx < len(waypoints):
        target = waypoints[current_waypoint_idx]
        dist_to_target = haversine_distance(lat, lon, target['lat'], target['lon'])
        
        # Check if we've passed the current waypoint
        passed_waypoint = False
        if dist_to_target < 15:  # meters
            passed_waypoint = True
        elif prev_dist is not None and dist_to_target > prev_dist and prev_dist < 200:
            passed_waypoint = True
        
        # Move to next waypoint if we've passed current one
        if passed_waypoint and current_waypoint_idx < len(waypoints) - 1:
            current_waypoint_idx += 1
            target = waypoints[current_waypoint_idx]
            dist_to_target = haversine_distance(lat, lon, target['lat'], target['lon'])
        
        # Calculate VMC to current target
        vmc = calculate_vmc(
            lat=np.array([lat]),
            lon=np.array([lon]),
            target_lat=target['lat'],
            target_lon=target['lon'],
            boat_speed=np.array([speed]),
            heading=np.array([heading])
        )
        
        boat_data.loc[idx, 'VMC_km_h'] = vmc[0]
        boat_data.loc[idx, 'target_waypoint'] = target['name']
        prev_dist = dist_to_target

# Summary statistics
vmc_valid = boat_data['VMC_km_h'].dropna()
print(f"\nVMC Statistics for {boat_code} in Race {race_number}:")
print(f"  Mean VMC: {vmc_valid.mean():.2f} km/h")
print(f"  Min VMC: {vmc_valid.min():.2f} km/h")
print(f"  Max VMC: {vmc_valid.max():.2f} km/h")
print(f"  Valid data points: {len(vmc_valid)} / {len(boat_data)}")

# Visualization: VMC over time
boat_data = boat_data.sort_values('DATETIME')
boat_data['time_seconds'] = (boat_data['DATETIME'] - boat_data['DATETIME'].iloc[0]).dt.total_seconds()

p = figure(title=f'VMC Over Time - {boat_code} (Race {race_number})', 
           x_axis_label='Time (seconds)', y_axis_label='VMC (km/h)',
           width=900, height=400, tools='pan,box_zoom,wheel_zoom,reset,save,hover')

source = ColumnDataSource(boat_data)
p.line('time_seconds', 'VMC_km_h', source=source, line_width=2, color='blue', legend_label='VMC')
p.circle('time_seconds', 'VMC_km_h', source=source, size=3, color='blue', alpha=0.5)

# Add zero line
from bokeh.models import Span
zero_line = Span(location=0, dimension='width', line_color='black', line_width=1, line_dash='dashed', line_alpha=0.5)
p.add_layout(zero_line)

p.legend.location = 'top_left'
p.grid.grid_line_alpha = 0.3
p.add_tools(HoverTool(tooltips=[
    ('Time', '@time_seconds{0.0}s'),
    ('VMC', '@VMC_km_h{0.2f} km/h'),
    ('Waypoint', '@target_waypoint')
]))

show(p)


# Detect tacks (significant heading changes) with persistence filtering
boat_data = boat_data.sort_values('DATETIME').reset_index(drop=True)
boat_data['tack'] = ''
boat_data['is_tack'] = False
boat_data['gust_direction'] = 0  # 1 = gust (increase), -1 = lull (decrease), 0 = no change
boat_data['head_of_lift'] = 0  # 1 = lift (favorable shift), -1 = header (unfavorable shift), 0 = no change

# First pass: determine tack for each point
for idx in range(len(boat_data)):
    row = boat_data.iloc[idx]
    
    if not pd.isna(row['HEADING_deg']) and not pd.isna(row['TWD_SGP_deg']):
        heading = row['HEADING_deg']
        twd = row['TWD_SGP_deg']
        
        # Calculate relative wind direction
        rel_wind = (heading - twd + 360) % 360
        
        if rel_wind < 180:
            current_tack = 'Port'
        else:
            current_tack = 'Starboard'
        
        boat_data.loc[idx, 'tack'] = current_tack

# Second pass: detect tack changes with persistence filtering
# Require tack to persist for at least 3 seconds (3 data points at 1Hz) before considering it valid
MIN_TACK_DURATION_SECONDS = 3  # Minimum time a tack must persist
MIN_TIME_BETWEEN_TACKS = 5  # Minimum time between valid tacks (seconds)

boat_data['time_seconds'] = (boat_data['DATETIME'] - boat_data['DATETIME'].iloc[0]).dt.total_seconds()
potential_tack_indices = []
current_tack_start = None
current_tack_type = None
last_valid_tack_time = -MIN_TIME_BETWEEN_TACKS - 1

for idx in range(len(boat_data)):
    row = boat_data.iloc[idx]
    current_tack = row['tack']
    current_time = row['time_seconds']
    
    if current_tack == '':
        continue
    
    if current_tack_type is None:
        # Initialize
        current_tack_type = current_tack
        current_tack_start = idx
    elif current_tack != current_tack_type:
        # Tack changed - check if previous tack persisted long enough
        tack_duration = boat_data.loc[current_tack_start:idx-1, 'time_seconds'].max() - \
                       boat_data.loc[current_tack_start:idx-1, 'time_seconds'].min()
        
        if tack_duration >= MIN_TACK_DURATION_SECONDS:
            # Previous tack was valid - mark the change point
            # Only mark if enough time has passed since last tack
            if current_time - last_valid_tack_time >= MIN_TIME_BETWEEN_TACKS:
                potential_tack_indices.append(idx)
                last_valid_tack_time = current_time
        
        # Start tracking new tack
        current_tack_type = current_tack
        current_tack_start = idx

# Mark valid tacks
for idx in potential_tack_indices:
    boat_data.loc[idx, 'is_tack'] = True

tack_count = len(potential_tack_indices)

# Detect gusts (wind speed changes)
prev_tws = None
for idx in range(len(boat_data)):
    row = boat_data.iloc[idx]
    if not pd.isna(row['TWS_SGP_km_h_1']):
        tws = row['TWS_SGP_km_h_1']
        if prev_tws is not None and not pd.isna(prev_tws):
            tws_change = tws - prev_tws
            # Gust: increase > 2 km/h, Lull: decrease > 2 km/h
            if tws_change > 2:
                boat_data.loc[idx, 'gust_direction'] = 1  # Gust
            elif tws_change < -2:
                boat_data.loc[idx, 'gust_direction'] = -1  # Lull
        prev_tws = tws

# Detect head of lift (wind direction shifts relative to boat heading)
prev_twd = None
prev_heading = None
for idx in range(len(boat_data)):
    row = boat_data.iloc[idx]
    if not pd.isna(row['TWD_SGP_deg']) and not pd.isna(row['HEADING_deg']):
        twd = row['TWD_SGP_deg']
        heading = row['HEADING_deg']
        
        # Calculate wind angle relative to boat heading
        wind_angle = (twd - heading + 360) % 360
        if wind_angle > 180:
            wind_angle = 360 - wind_angle
        
        if prev_twd is not None and prev_heading is not None:
            prev_wind_angle = (prev_twd - prev_heading + 360) % 360
            if prev_wind_angle > 180:
                prev_wind_angle = 360 - prev_wind_angle
            
            # Lift: wind shifts toward the bow (reduces wind angle)
            # Header: wind shifts away from the bow (increases wind angle)
            angle_change = wind_angle - prev_wind_angle
            
            # For upwind: lift = wind shifts toward bow (positive for port, negative for starboard)
            # For downwind: opposite
            # Simplified: if wind angle decreases, it's a lift (more favorable)
            if abs(angle_change) > 2:  # Significant shift (> 2 degrees)
                if angle_change < 0:
                    boat_data.loc[idx, 'head_of_lift'] = 1  # Lift (favorable)
                else:
                    boat_data.loc[idx, 'head_of_lift'] = -1  # Header (unfavorable)
        
        prev_twd = twd
        prev_heading = heading

print(f"  Detected {tack_count} tacks")

from bokeh.layouts import column
from bokeh.models import Label

source2 = ColumnDataSource(boat_data)
time_range = (boat_data['time_seconds'].min(), boat_data['time_seconds'].max())

# Panel 1: VMC with Tacks
p_vmc = figure(
    title=f'VMC with Tacks - {boat_code} (Race {race_number})',
    x_axis_label='Time (seconds)',
    y_axis_label='VMC (km/h)',
    width=1000,
    height=300,
    tools='pan,box_zoom,wheel_zoom,reset,save,hover',
    x_range=time_range
)

# Plot VMC line
p_vmc.line('time_seconds', 'VMC_km_h', source=source2, line_width=2, color='#1f77b4', legend_label='VMC')

# Mark tacks with prominent vertical lines
tack_times = boat_data[boat_data['is_tack']]['time_seconds'].values
for tack_time in tack_times:
    tack_line = Span(location=tack_time, dimension='height', line_color='red', line_width=3, line_dash='dashed', line_alpha=0.8)
    p_vmc.add_layout(tack_line)

# Add tack labels
for i, tack_time in enumerate(tack_times):
    tack_idx = boat_data[boat_data['time_seconds'] == tack_time].index
    if len(tack_idx) > 0:
        tack_name = boat_data.loc[tack_idx[0], 'tack']
        label = Label(x=tack_time, y=boat_data['VMC_km_h'].max() * 0.9,
                     text=f"Tack: {tack_name}", text_font_size='8pt',
                     text_color='red', text_alpha=0.8, x_offset=5)
        p_vmc.add_layout(label)

# Add zero line
zero_line = Span(location=0, dimension='width', line_color='black', line_width=1, line_dash='dotted', line_alpha=0.5)
p_vmc.add_layout(zero_line)

p_vmc.legend.location = 'top_left'
p_vmc.grid.grid_line_alpha = 0.3
p_vmc.add_tools(HoverTool(tooltips=[
    ('Time', '@time_seconds{0.0}s'),
    ('VMC', '@VMC_km_h{0.2f} km/h'),
    ('Tack', '@tack')
]))

# Panel 2: Wind Speed with Gusts/Lulls
p_wind_speed = figure(
    title='Wind Speed with Gusts & Lulls',
    x_axis_label='Time (seconds)',
    y_axis_label='Wind Speed (km/h)',
    width=1000,
    height=250,
    tools='pan,box_zoom,wheel_zoom,reset,save,hover',
    x_range=time_range
)

# Plot wind speed line
p_wind_speed.line('time_seconds', 'TWS_SGP_km_h_1', source=source2, 
                  line_width=2, color='#2ca02c', legend_label='Wind Speed', alpha=0.7)

# Mark gusts (wind speed increase)
gust_data = boat_data[boat_data['gust_direction'] == 1].copy()
if len(gust_data) > 0:
    p_wind_speed.circle(gust_data['time_seconds'], gust_data['TWS_SGP_km_h_1'],
                        size=12, color='#00ff00', alpha=0.8, legend_label='Gust', line_width=2, line_color='darkgreen')

# Mark lulls (wind speed decrease)
lull_data = boat_data[boat_data['gust_direction'] == -1].copy()
if len(lull_data) > 0:
    p_wind_speed.circle(lull_data['time_seconds'], lull_data['TWS_SGP_km_h_1'],
                        size=12, color='#ff8c00', alpha=0.8, legend_label='Lull', line_width=2, line_color='darkorange')

# Add tack markers to wind speed plot too
for tack_time in tack_times:
    tack_line = Span(location=tack_time, dimension='height', line_color='red', line_width=2, line_dash='dashed', line_alpha=0.5)
    p_wind_speed.add_layout(tack_line)

p_wind_speed.legend.location = 'top_left'
p_wind_speed.grid.grid_line_alpha = 0.3
p_wind_speed.add_tools(HoverTool(tooltips=[
    ('Time', '@time_seconds{0.0}s'),
    ('Wind Speed', '@TWS_SGP_km_h_1{0.1f} km/h')
]))

# Panel 3: Wind Direction with Lifts/Headers
p_wind_dir = figure(
    title='Wind Direction with Lifts & Headers',
    x_axis_label='Time (seconds)',
    y_axis_label='Wind Direction (degrees)',
    width=1000,
    height=250,
    tools='pan,box_zoom,wheel_zoom,reset,save,hover',
    x_range=time_range
)

# Plot wind direction line
p_wind_dir.line('time_seconds', 'TWD_SGP_deg', source=source2,
                line_width=2, color='#9467bd', legend_label='Wind Direction', alpha=0.7)

# Mark lifts (favorable wind shift)
lift_data = boat_data[boat_data['head_of_lift'] == 1].copy()
if len(lift_data) > 0:
    p_wind_dir.triangle(lift_data['time_seconds'], lift_data['TWD_SGP_deg'],
                        size=14, color='#1f77b4', alpha=0.8, legend_label='Lift', line_width=2, line_color='darkblue')

# Mark headers (unfavorable wind shift)
header_data = boat_data[boat_data['head_of_lift'] == -1].copy()
if len(header_data) > 0:
    p_wind_dir.triangle(header_data['time_seconds'], header_data['TWD_SGP_deg'],
                        size=14, color='#d62728', alpha=0.8, legend_label='Header', line_width=2, line_color='darkred')

# Add tack markers to wind direction plot too
for tack_time in tack_times:
    tack_line = Span(location=tack_time, dimension='height', line_color='red', line_width=2, line_dash='dashed', line_alpha=0.5)
    p_wind_dir.add_layout(tack_line)

p_wind_dir.legend.location = 'top_left'
p_wind_dir.grid.grid_line_alpha = 0.3
p_wind_dir.add_tools(HoverTool(tooltips=[
    ('Time', '@time_seconds{0.0}s'),
    ('Wind Direction', '@TWD_SGP_deg{0.1f}°'),
    ('Tack', '@tack')
]))

# Combine all panels
combined_chart = column(p_vmc, p_wind_speed, p_wind_dir)
show(combined_chart)

Processing Race 25011905
Course has 8 waypoints: ['SL2', 'M1', 'LG2', 'WG2', 'LG2', 'WG2', 'LG1', 'FL2']

Boat BRA: 687 data points for race 25011905

VMC Statistics for BRA in Race 25011905:
  Mean VMC: 23.98 km/h
  Min VMC: -60.70 km/h
  Max VMC: 80.45 km/h
  Valid data points: 687 / 687


  Detected 9 tacks


## Question 3: Verify and comment on the boats calibration. If possible propose a post-calibrated set of wind numbers and a potential calibration table.


## Question 4: Given a timeseries of Lat/Lon positions and a course XML, in a Python notebook, calculate a Distance to Leader metric for each boat.

In [144]:
# Load all boat data once
boat_logs_all = load_boat_logs(data_dir='Data')

# Get all race XML files
race_xml_dir = Path('Data/Race_XMLs')
race_xml_files = sorted(race_xml_dir.glob('*.xml'))

# Process each race
for xml_path in race_xml_files:
    # Extract race number from filename
    race_number = int(xml_path.stem.split('_')[0])
    print(f"Processing Race {race_number}")
    
    # Load course XML
    course_data = parse_course_xml(xml_path)
    
    # Get finish line (last waypoint)
    mark_sequence = course_data['mark_sequence']
    marks_by_compound = {m['compound_id']: m for m in course_data['marks']}
    waypoints = []
    for corner in mark_sequence:
        compound_id = corner['compound_mark_id']
        if compound_id in marks_by_compound:
            waypoints.append(marks_by_compound[compound_id])
    finish_line = waypoints[-1] if waypoints else None
    
    if not finish_line:
        print(f"  Skipping race {race_number}: No finish line found")
        continue
    
    # Filter boat data to this race
    boat_logs = {}
    all_times = set()
    
    for boat_code, df in boat_logs_all.items():
        df = df.copy()
        df['DATETIME'] = pd.to_datetime(df['DATETIME'])
        df_race = df[df['TRK_RACE_NUM_unk'] == race_number].copy()
        if len(df_race) > 0:
            boat_logs[boat_code] = df_race
            all_times.update(df_race['DATETIME'].values)
    
    if len(boat_logs) < 2:
        print(f"  Skipping race {race_number}: Not enough boats ({len(boat_logs)})")
        continue
    
    print(f"  Found {len(boat_logs)} boats")
    
    # Calculate distance to leader for each timestamp
    results = []
    common_times = sorted(list(all_times))
    
    for time in common_times:
        boat_positions = {}
        boat_distances_to_finish = {}
        
        # Get position of each boat at this time
        for boat_code, df in boat_logs.items():
            time_match = df[df['DATETIME'] == time]
            if len(time_match) > 0:
                lat = time_match.iloc[0]['LATITUDE_GPS_unk']
                lon = time_match.iloc[0]['LONGITUDE_GPS_unk']
                boat_positions[boat_code] = {'lat': lat, 'lon': lon}
                boat_distances_to_finish[boat_code] = haversine_distance(
                    lat, lon, finish_line['lat'], finish_line['lon']
                )
        
        if len(boat_positions) < 2:
            continue
        
        # Leader is closest to finish
        leader = min(boat_distances_to_finish.items(), key=lambda x: x[1])[0]
        leader_pos = boat_positions[leader]
        
        # Calculate distance to leader for each boat
        for boat_code, pos in boat_positions.items():
            dist_to_leader = 0.0 if boat_code == leader else haversine_distance(
                pos['lat'], pos['lon'], leader_pos['lat'], leader_pos['lon']
            )
            results.append({
                'time': time,
                'boat': boat_code,
                'distance_to_leader_m': dist_to_leader
            })
    
    # Create DataFrame
    distance_df = pd.DataFrame(results)
    if len(distance_df) == 0:
        print(f"  Skipping race {race_number}: No data points")
        continue
    
    # Prepare for visualization
    distance_df['time'] = pd.to_datetime(distance_df['time'])
    distance_df = distance_df.sort_values('time')
    distance_df['time_seconds'] = (distance_df['time'] - distance_df['time'].iloc[0]).dt.total_seconds()
    
    # Create simple line chart for each boat
    p = figure(
        title=f'Distance to Leader - Race {race_number}',
        x_axis_label='Time (seconds)',
        y_axis_label='Distance to Leader (meters)',
        width=900,
        height=500,
        tools='pan,box_zoom,wheel_zoom,reset,save,hover'
    )
    
    # Plot each boat
    from bokeh.palettes import Category10
    boats = sorted(distance_df['boat'].unique())
    colors = Category10[10] * ((len(boats) // 10) + 1)
    
    for i, boat in enumerate(boats):
        boat_data = distance_df[distance_df['boat'] == boat]
        source = ColumnDataSource(boat_data)
        p.line('time_seconds', 'distance_to_leader_m', source=source,
               line_width=2, color=colors[i], legend_label=boat)
    
    p.legend.location = 'top_left'
    p.grid.grid_line_alpha = 0.3
    
    show(p)


Processing Race 25011905
  Found 11 boats


Processing Race 25011906
  Found 11 boats


Processing Race 25011907
  Found 11 boats


Processing Race 25011908
  Found 3 boats


## Question 5: Given a course XML, along with a wind speed and direction and a polar, calculate the minimum number of tacks or gybes for each leg of the course and each gate mark on the leg.

## Question 6: Calculate a “tacked” set of variables depending on the tack of the boat, so that sailors don’t need to think about what tack they’re on when looking at measurements. And show the results in a visualisation.


## Question 7: Given a set of tacks (in CSV), and train a model to explain the key features of these tacks when optimizing for vmg. Show appropriate visualisations to explain your conclusions.

## Question 8: Give insights on the racing on what made a team win or underperform in the race.